# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.


In [52]:
# YOUR CHANGES HERE

prices = pd.read_csv('strawberry-prices.tsv', sep='\t')

train = prices[0:48]
test = prices[48:]
train = train.rename(mapper={'month':'ds', 'price':'y'}, axis=1)

prices_model = Prophet()
prices_model.fit(train)

prices_future = prices_model.make_future_dataframe(periods=12, freq='MS')
future_2024 = prices_future[prices_future['ds'].dt.year == 2024]
prices_forecast = prices_model.predict(future_2024)
strawberry_backtest = prices_forecast[['ds', 'yhat']].tail(12)
strawberry_backtest = strawberry_backtest.rename(columns={'ds': 'month', 'yhat':'price'})

06:44:28 - cmdstanpy - INFO - Chain [1] start processing


06:44:28 - cmdstanpy - INFO - Chain [1] done processing


In [53]:
strawberry_backtest.to_csv('strawberry-backtest.tsv', sep='\t')

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth? (If your mean is not close to zero, then you may be missing a long term trend.)

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [54]:
# YOUR CHANGES HERE

import statistics

test = test.reset_index(drop=True)

residuals = test['price'] - strawberry_backtest['price']
mean = np.mean(residuals)
std = statistics.stdev(residuals)
print(f'Mean: {mean}')
print(f'STD: {std}')

Mean: -0.03477350807363696
STD: 0.274374327235493


In [55]:
backtest_accuracy = {'mean': mean, 'std': std}
backtest_accuracy = pd.DataFrame(backtest_accuracy, index=[0])
backtest_accuracy.to_csv('backtest-accuracy.tsv', sep='\t')

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.

In [63]:
# YOUR CHANGES HERE

prices = prices.rename(mapper={'month':'ds', 'price':'y'}, axis=1)

model = Prophet()
model.fit(prices)

model_future = model.make_future_dataframe(periods=12, freq='MS')
future_2025 = model_future[model_future['ds'].dt.year == 2025]
model_forecast = model.predict(future_2025)
strawberry_forecast = model_forecast[['ds', 'yhat']].tail(12)
strawberry_forecast = strawberry_forecast.rename(columns={'ds': 'month', 'yhat':'price'})
strawberry_forecast

06:47:32 - cmdstanpy - INFO - Chain [1] start processing


06:47:32 - cmdstanpy - INFO - Chain [1] done processing


,month,price
0,2025-01-01,4.227590
1,2025-02-01,4.139785
2,2025-03-01,3.625474
3,2025-04-01,3.621645
4,2025-05-01,3.334506
5,2025-06-01,3.056036
6,2025-07-01,3.009510
7,2025-08-01,3.263252
8,2025-09-01,3.399879
9,2025-10-01,3.662965


In [57]:
strawberry_forecast.to_csv('strawberry-forecast.tsv', sep='\t')

Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [72]:
# YOUR CHANGES HERE

pairs = []
for i, buy_row in strawberry_forecast.iterrows():
    for j, sell_row in strawberry_forecast.iterrows():
        if j > i:
            months = j - i
            per_pint = (sell_row.price * 0.9) - buy_row.price - 0.20 - (0.10 * months)
            pints = int(1_000_000 // buy_row.price)
            pairs.append([buy_row.month, sell_row.month, pints, pints*per_pint])

pairs_df = pd.DataFrame(pairs, columns=['buy_month', 'sell_month', 'pints_purchased', 'expected_profit'])
pd.set_option('display.max_rows', None)
pairs_df


,buy_month,sell_month,pints_purchased,expected_profit
0,2025-01-01,2025-02-01,236541,-189654.538921
1,2025-01-01,2025-03-01,236541,-322798.744760
2,2025-01-01,2025-04-01,236541,-347268.071306
3,2025-01-01,2025-05-01,236541,-432050.176875
4,2025-01-01,2025-06-01,236541,-514986.971325
5,2025-01-01,2025-07-01,236541,-548545.756160
6,2025-01-01,2025-08-01,236541,-518181.592126
7,2025-01-01,2025-09-01,236541,-512749.542080
8,2025-01-01,2025-10-01,236541,-480396.080618
9,2025-01-01,2025-11-01,236541,-399365.649982


In [73]:
pairs_df.to_csv('timings.tsv', sep='\t', index=False)

Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns best_profit and one_std_profit.

In [67]:
# YOUR CHANGES HERE

pairs_df.sort_values(by=['expected_profit'])

,buy_month,sell_month,pints_purchased,expected_profit
5,2025-01-01,2025-07-01,28757524.0,-6.668957e+07
6,2025-01-01,2025-08-01,28757524.0,-6.299804e+07
4,2025-01-01,2025-06-01,28757524.0,-6.260965e+07
7,2025-01-01,2025-09-01,28757524.0,-6.233764e+07
15,2025-02-01,2025-07-01,28757524.0,-6.128878e+07
8,2025-01-01,2025-10-01,28757524.0,-5.840426e+07
16,2025-02-01,2025-08-01,28757524.0,-5.759725e+07
14,2025-02-01,2025-06-01,28757524.0,-5.720886e+07
17,2025-02-01,2025-09-01,28757524.0,-5.693685e+07
18,2025-02-01,2025-10-01,28757524.0,-5.300347e+07


In [68]:
best_profit = pairs_df['expected_profit'][55]
one_std_profit = best_profit - std

check = pd.DataFrame([[best_profit, one_std_profit]], columns = ['best_profit', 'one_std_profit'])
check.to_csv('check.tsv', sep='\t')

Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.